In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module): ##super와 nn.Module이 같이 상속의 개념
    def __init__(self):
        super().__init__() ##상속 호출

        self.flatten = nn.Flatten()
        
        self.linear_relu_stack = nn.Sequential(
            
            nn.Linear(28*28, 512),
            nn.ReLU(),

            nn.Linear(512, 512),
            nn.ReLU(),

            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device) ##cpu에서 게산이 되는데 사용하면 
print(model)                       ##예를 들어 GPU메모리로 보내줄 수 있음 

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) ##6만개 샘플
    
    model.train()
    correct = 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) 

        # Compute prediction error
        pred = model(X) ##model = NeuralNetwork().to(device)
        loss = loss_fn(pred, y) ##loss_fn = nn.CrossEntropyLoss()

        # Backpropagation
        loss.backward() ##역전파(backpropagation) 단계입니다. 
        optimizer.step() ##역전파 단계에서 계산된 기울기를 사용하여 모델의 매개변수를 조정 
        optimizer.zero_grad() ##매 반복마다 기존의 기울기를 초기화합니다. 

        correct += (pred.argmax(1) == y).type(torch.float).sum().item() 
        ##불리언(bool) 값을 부동 소수점 형태로 변환
        ##각 배치에 대해 일치하는 예측값의 개수를 합산
        ##텐서의 값을 Python의 스칼라 값으로 변환
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X) ##batch는 index
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    accuracy = correct / size
    return accuracy, loss

In [ ]:
a=1.23456
print(f"{a:f}")
print(f"{a:10f}")
print(f"{a:<10f}")

1.234560
  1.234560
1.234560  


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    
    print(f"Test Error: \n Accuracy: {(100 * accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return accuracy

In [ ]:
epochs = 5

train_acc_list = []
test_acc_list = []

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_acc = test(test_dataloader, model, loss_fn)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
print("Done!")

Epoch 1
-------------------------------
loss: 2.313851  [   64/60000]
loss: 2.160453  [ 6464/60000]
loss: 1.780432  [12864/60000]
loss: 1.478305  [19264/60000]
loss: 1.132754  [25664/60000]
loss: 1.038851  [32064/60000]
loss: 0.997471  [38464/60000]
loss: 0.864540  [44864/60000]
loss: 0.865240  [51264/60000]
loss: 0.808004  [57664/60000]
Test Error: 
 Accuracy: 71.8%, Avg loss: 0.785318 

Epoch 2
-------------------------------
loss: 0.778429  [   64/60000]
loss: 0.853303  [ 6464/60000]
loss: 0.576580  [12864/60000]
loss: 0.773969  [19264/60000]
loss: 0.660517  [25664/60000]
loss: 0.632665  [32064/60000]
loss: 0.712619  [38464/60000]
loss: 0.682149  [44864/60000]
loss: 0.703607  [51264/60000]
loss: 0.635477  [57664/60000]
Test Error: 
 Accuracy: 77.8%, Avg loss: 0.626486 

Epoch 3
-------------------------------
loss: 0.550261  [   64/60000]
loss: 0.659717  [ 6464/60000]
loss: 0.432075  [12864/60000]
loss: 0.655386  [19264/60000]
loss: 0.580500  [25664/60000]
loss: 0.557282  [32064/600

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad(): ##노 그레디언트: 이 부분은 그 아래의 코드가 추론을 위한 것임을 나타냅니다. 
                      ##따라서 기울기 계산을 비활성화합니다. 
                      ##추론 시에는 역전파가 필요하지 않으므로 메모리를 절약하기 위해 사용됩니다.
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
